In [19]:
import mlflow
import mlflow.sklearn

# Configurer MLflow pour utiliser le répertoire local
mlflow.set_experiment("my_experiment")


<Experiment: artifact_location='file:///C:/Users/adsbouzidia/Desktop/EPSI_cours_tp/PROJET/1_Rendu_MSPR_Data/mlruns/926019719233567901', creation_time=1726435362776, experiment_id='926019719233567901', last_update_time=1726435362776, lifecycle_stage='active', name='my_experiment', tags={}>

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

def train_and_evaluate(X_train, X_test, y_train, y_test, features, model_params):
    # Créer le pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', StandardScaler()),
        ('regressor', RandomForestRegressor(**model_params))
    ])
    
    # Entraîner le modèle
    pipeline.fit(X_train[features], y_train)
    
    # Prédictions
    y_pred = pipeline.predict(X_test[features])
    
    # Calculer les métriques
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return pipeline, mae, r2


# Charger les données des elections

In [5]:
# Charger les données
alldata = pd.read_csv('données/Nettoyées_Et_Modifiées/Elections/Elections_12_17_22.csv', sep=';', encoding='ISO-8859-1')
# Filtrer les données pour les années 2017 et 2022
data = alldata[alldata['Annee'].isin([2017, 2022])]

# Afficher les premières lignes pour vérifier
print(data.head())


       Annee  Tour  Nombre_candidat  Code_du_departement  \
16788   2017     1               11                   75   
16789   2017     1               11                   75   
16790   2017     1               11                   77   
16791   2017     1               11                   77   
16792   2017     1               11                   77   

      Libelle_du_departement  Code_de_la_commune Libelle_de_la_commune  \
16788                  Paris                  56                 Paris   
16789                  Paris                  56                 Paris   
16790         Seine-et-Marne                   1      Achères-la-Forêt   
16791         Seine-et-Marne                   1      Achères-la-Forêt   
16792         Seine-et-Marne                   2               Amillis   

       Inscrits  Abstentions  Nombre_votants  Blancs_et_nulls Nom_candidat  \
16788   1301637       210200         1091437            14878       macron   
16789   1301637       210200         1

# Encodage des variables catégorielles

In [10]:
# Encodage des variables catégorielles
data_encoded = pd.get_dummies(data, columns=['Sexe', 'Orientation_politique'], drop_first=True)

In [12]:
data_encoded.columns

Index(['Annee', 'Tour', 'Nombre_candidat', 'Code_du_departement',
       'Libelle_du_departement', 'Code_de_la_commune', 'Libelle_de_la_commune',
       'Inscrits', 'Abstentions', 'Nombre_votants', 'Blancs_et_nulls',
       'Nom_candidat', 'Age', 'Nombre_voix', 'Parti_politique', 'Sexe_H',
       'Orientation_politique_Droite',
       'Orientation_politique_Droite souverainiste',
       'Orientation_politique_Extreme droite',
       'Orientation_politique_Extreme gauche',
       'Orientation_politique_ExtrÃªme droite', 'Orientation_politique_Gauche',
       'Orientation_politique_Gauche radicale',
       'Orientation_politique_Ãcologie'],
      dtype='object')

In [15]:
data_encoded

,Annee,Tour,Nombre_candidat,Code_du_departement,Libelle_du_departement,Code_de_la_commune,Libelle_de_la_commune,Inscrits,Abstentions,Nombre_votants,...,Parti_politique,Sexe_H,Orientation_politique_Droite,Orientation_politique_Droite souverainiste,Orientation_politique_Extreme droite,Orientation_politique_Extreme gauche,Orientation_politique_ExtrÃªme droite,Orientation_politique_Gauche,Orientation_politique_Gauche radicale,Orientation_politique_Ãcologie
16788,2017,1,11,75,Paris,56,Paris,1301637,210200,1091437,...,En Marche!,True,False,False,False,False,False,False,False,False
16789,2017,1,11,75,Paris,56,Paris,1301637,210200,1091437,...,Les Republicains,True,True,False,False,False,False,False,False,False
16790,2017,1,11,77,Seine-et-Marne,1,Achères-la-Forêt,886,134,752,...,Les Republicains,True,True,False,False,False,False,False,False,False
16791,2017,1,11,77,Seine-et-Marne,1,Achères-la-Forêt,886,134,752,...,En Marche!,True,False,False,False,False,False,False,False,False
16792,2017,1,11,77,Seine-et-Marne,2,Amillis,568,78,490,...,Front National,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39643,2022,2,2,95,Val-d'Oise,680,Villiers-le-Bel,12414,5099,7315,...,Parti Communiste FranÃ§ais,True,False,False,False,False,False,True,False,False
39644,2022,2,2,95,Val-d'Oise,682,Villiers-le-Sec,110,26,84,...,La RÃ©publique En Marche,True,False,False,False,False,False,False,False,False
39645,2022,2,2,95,Val-d'Oise,682,Villiers-le-Sec,110,26,84,...,Parti Communiste FranÃ§ais,True,False,False,False,False,False,True,False,False
39646,2022,2,2,95,Val-d'Oise,690,Wy-dit-Joli-Village,285,53,232,...,La RÃ©publique En Marche,True,False,False,False,False,False,False,False,False


In [18]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split

# Encodage des variables catégorielles
data_encoded = pd.get_dummies(data, columns=['Sexe', 'Orientation_politique'], drop_first=True)

# Initialisation des paramètres du modèle
model_params = {'n_estimators': 100, 'max_depth': None, 'random_state': 42}

# Définir la fonction d'entraînement et d'évaluation
def train_and_evaluate(X_train, X_test, y_train, y_test, features, model_params):
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_absolute_error, r2_score

    # Création du pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', StandardScaler()),
        ('regressor', RandomForestRegressor(**model_params))
    ])

    # Entraîner le modèle
    pipeline.fit(X_train[features], y_train)

    # Prédictions
    y_pred = pipeline.predict(X_test[features])

    # Calcul des métriques
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return pipeline, mae, r2

# Préparer les données
y = data_encoded['Nombre_voix']

# Boucle d'expérimentation avec différentes combinaisons de features
new_features_sets = [
    ['Age'],
    ['Age', 'Sexe_H'],  # Encodé par get_dummies
    ['Age', 'Sexe_H', 'Orientation_politique_Droite'],  # Ajout de plus de caractéristiques
    # Ajouter plus de jeux de caractéristiques ici
]

for features in new_features_sets:
    X = data_encoded[features]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    with mlflow.start_run():
        # Entraîner et évaluer le modèle
        model, mae, r2 = train_and_evaluate(X_train, X_test, y_train, y_test, features, model_params)
        
        # Log des paramètres et des métriques
        mlflow.log_param('features', features)
        mlflow.log_param('model_params', model_params)
        mlflow.log_metric('mean_absolute_error', mae)
        mlflow.log_metric('r2_score', r2)
        
        # Log du modèle
        mlflow.sklearn.log_model(model, 'model')

        # Afficher le score pour vérification
        print(f'Features: {features}, MAE: {mae}, R²: {r2}')


2024/09/16 00:07:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Features: ['Age'], MAE: 995.3413859146495, R²: 0.005919116733869245


2024/09/16 00:07:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Features: ['Age', 'Sexe_H'], MAE: 990.9562312331766, R²: 0.005918028417784216


2024/09/16 00:07:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Features: ['Age', 'Sexe_H', 'Orientation_politique_Droite'], MAE: 985.6900393890237, R²: 0.005902856546432966


In [21]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

# Préparer les données encodées
y = data_encoded['Nombre_voix']

# Boucle d'expérimentation avec différentes combinaisons de features
new_features_sets = [
    ['Age'],
    ['Age', 'Sexe_H'],  # Encodé par get_dummies
    ['Age', 'Sexe_H', 'Orientation_politique_Droite'],  # Ajout de plus de caractéristiques
    # Ajouter plus de jeux de caractéristiques ici
]

for features in new_features_sets:
    X = data_encoded[features]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    with mlflow.start_run():
        # Entraîner et évaluer le modèle
        model, mae, r2 = train_and_evaluate(X_train, X_test, y_train, y_test, features, model_params)
        
        # Log des paramètres et des métriques
        mlflow.log_param('features', features)
        mlflow.log_param('model_params', model_params)
        mlflow.log_metric('mean_absolute_error', mae)
        mlflow.log_metric('r2_score', r2)
        
        # Créer un input example (première ligne de X_train)
        input_example = X_train.head(1)
        
        # Inférer la signature du modèle
        signature = infer_signature(X_train, model.predict(X_train))
        
        # Log du modèle avec un input example et la signature
        mlflow.sklearn.log_model(model, 'model', signature=signature, input_example=input_example)

        # Afficher le score pour vérification
        print(f'Features: {features}, MAE: {mae}, R²: {r2}')


Features: ['Age'], MAE: 995.3413859146495, R²: 0.005919116733869245
Features: ['Age', 'Sexe_H'], MAE: 990.9562312331766, R²: 0.005918028417784216
Features: ['Age', 'Sexe_H', 'Orientation_politique_Droite'], MAE: 985.6900393890237, R²: 0.005902856546432966
